In [19]:
import pandas as pd
import psycopg2

DB_CONFIG = {
    "dbname": "real_state_dwh",
    "user": "admin",
    "password": "admin",
    "host": "postgres_general", 
    "port": 5432
}

print("Connecting to Postgres...")
try:
    conn = psycopg2.connect(**DB_CONFIG)
    query = "SELECT * FROM ml_data_mart"
    df = pd.read_sql(query, conn)
    conn.close()
    print(f"Loaded {len(df)} rows")
except Exception as e:
    print(f"Error: {e}")

Connecting to Postgres...
Loaded 13244 rows


/tmp/ipykernel_21464/2801969052.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [20]:
import pandas as pd
import numpy as np
import pickle

# Load data (assuming df is already loaded from previous cell)
print("Building Analytics Model...")

# Clean data
df_analytics = df.copy()
df_analytics = df_analytics.dropna(subset=['city', 'price', 'area'])
df_analytics = df_analytics[(df_analytics['area'] > 20) & (df_analytics['price'] > 50000)]

# Property Type Cleaning
def clean_type(t):
    t = str(t).lower()
    if 'studio' in t: return 'Studio'
    if 'duplex' in t: return 'Duplex'
    if 'penthouse' in t: return 'Penthouse'
    if 'chalet' in t: return 'Chalet'
    if 'townhouse' in t: return 'Townhouse'
    if 'twin' in t: return 'Twin House'
    if any(x in t for x in ['villa', 'stand', 'mansion', 'palace']): return 'Villa'
    return 'Apartment'

df_analytics['type_clean'] = df_analytics['property_type'].apply(clean_type)

#Calculate Analytics by City & Region
analytics_data = {}

for city in df_analytics['city'].unique():
    city_data = df_analytics[df_analytics['city'] == city]
    
    # Calculate City Average Price (The benchmark for this city)
    city_avg_price = city_data['price'].mean()
    
    analytics_data[city] = {
        'total_properties': len(city_data),
        'avg_price': city_avg_price,
        'median_price': city_data['price'].median(),
        'min_price': city_data['price'].min(),
        'max_price': city_data['price'].max(),
        'avg_area': city_data['area'].mean(),
        'top_property_type': city_data['type_clean'].mode()[0] if not city_data['type_clean'].mode().empty else 'N/A',
        'property_type_distribution': city_data['type_clean'].value_counts().to_dict(),
        'regions': {}
    }
    
    # Per Region
    for region in city_data['region'].dropna().unique():
        region_data = city_data[city_data['region'] == region]
        
        if len(region_data) < 3:  # Skip regions with too few properties
            continue
        
        # Hotspot/Coldspot logic (STRICTLY BASED ON CITY AVERAGE)
        region_avg_price = region_data['price'].mean()
        
        #Hotspot & coldspot
        if region_avg_price > city_avg_price * 1.2:
            status = 'Hotspot'
        elif region_avg_price < city_avg_price * 0.8:
            status = 'Coldspot' 
        else:
            status = 'Average'
        
        analytics_data[city]['regions'][region] = {
            'total_properties': len(region_data),
            'avg_price': region_avg_price,
            'median_price': region_data['price'].median(),
            'avg_area': region_data['area'].mean(),
            'top_property_type': region_data['type_clean'].mode()[0] if not region_data['type_clean'].mode().empty else 'N/A',
            'property_type_distribution': region_data['type_clean'].value_counts().to_dict(),
            'status': status
        }

# Save Analytics Model
with open('analytics_model.pkl', 'wb') as f:
    pickle.dump({
        'analytics_data': analytics_data,
        'df': df_analytics
    }, f)

print(f"Analytics Model saved with City-Relative Logic!")

Building Analytics Model...
Analytics Model saved with City-Relative Logic!


In [21]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd
import pickle

# --- CLEAN DATA ---
print("Cleaning data...")
df_clean = df.drop_duplicates(subset=['price', 'area', 'city', 'region']).copy()
df_clean = df_clean[(df_clean['area'] > 20) & (df_clean['price'] > 50000)]

# Remove inf/NaN values
df_clean = df_clean.replace([np.inf, -np.inf], np.nan)

# Fill numeric columns
numeric_cols = ['area', 'bedrooms', 'bathrooms', 'price', 'latitude', 'longitude']
for col in numeric_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# Fill text columns
df_clean['title'] = df_clean['title'].fillna('')
df_clean['description'] = df_clean['description'].fillna('')
df_clean['region'] = df_clean['region'].fillna('')
df_clean['text_features'] = (df_clean['title'] + " " + df_clean['description']).str.lower()

# Property Type Cleaning
def clean_type(t):
    t = str(t).lower()
    if 'studio' in t: return 'Studio'
    if 'duplex' in t: return 'Duplex'
    if 'penthouse' in t: return 'Penthouse'
    if 'chalet' in t: return 'Chalet'
    if 'townhouse' in t: return 'Townhouse'
    if 'twin' in t: return 'Twin House'
    if any(x in t for x in ['villa', 'stand', 'mansion', 'palace']): return 'Villa'
    return 'Apartment'

df_clean['type_clean'] = df_clean['property_type'].apply(clean_type)
df_clean = df_clean.reset_index(drop=True)

print(f"Cleaned data: {len(df_clean)} rows")

# --- AMENITIES ---
amenity_cols = ['jacuzzi','garden','balcony','pool','parking','gym','maids_quarters','spa']
for col in amenity_cols:
    df_clean[col] = df_clean.get(col, 0).fillna(0).astype(int)

# --- WEIGHTED FEATURES (NO CITY - will filter first) ---
print("Creating weighted features...")

# Property Type Weight (×3)
type_weight = 3
for i in range(type_weight):
    df_clean[f'type_{i}'] = df_clean['type_clean']

# Price Weight (×3)
price_weight = 3
for i in range(price_weight):
    df_clean[f'price_{i}'] = df_clean['price']

# Area Weight (×2)
area_weight = 2
for i in range(area_weight):
    df_clean[f'area_{i}'] = df_clean['area']

# Beds/Bath Weight (×2)
bed_bath_weight = 2
for i in range(bed_bath_weight):
    df_clean[f'bed_{i}'] = df_clean['bedrooms']
    df_clean[f'bath_{i}'] = df_clean['bathrooms']

# --- FEATURE SETUP (NO CITY) ---
numeric_features = [] + amenity_cols
categorical_features = []

# Add weighted features
categorical_features += [f'type_{i}' for i in range(type_weight)]
numeric_features += [f'price_{i}' for i in range(price_weight)]
numeric_features += [f'area_{i}' for i in range(area_weight)]
numeric_features += [f'bed_{i}' for i in range(bed_bath_weight)]
numeric_features += [f'bath_{i}' for i in range(bed_bath_weight)]

text_features = 'text_features'

print(f"Features: {len(numeric_features)} numeric, {len(categorical_features)} categorical")

# --- PREPROCESSING PIPELINE ---
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ('text', TfidfVectorizer(max_features=100, stop_words='english'), text_features)
])

print("Transforming features...")
X_processed = preprocessor.fit_transform(df_clean)
print(f"Transformed shape: {X_processed.shape}")

# --- TRAIN MODEL ---
print("Training KNN model...")
model = NearestNeighbors(n_neighbors=100, metric='cosine')
model.fit(X_processed)
print("Model trained!")

# --- MATCH SCORE FUNCTION (NO CITY - already filtered) ---
def calculate_match_score(target, result, selected_amenity=None):
    """
    Scoring AFTER city filter:
    
    NO Preference:
    - Budget: 30%
    - Type: 25%
    - Area: 15%
    - Bath: 15%
    - Bed: 15%
    
    WITH Preference:
    - Budget: 30%
    - Type: 20%
    - Preference: 20%
    - Area: 10%
    - Bath: 10%
    - Bed: 10%
    """
    score = 0
    has_preference = bool(selected_amenity and selected_amenity.strip())
    
    # Budget (30%)
    price_diff = abs(target['price'] - result['price']) / target['price'] if target['price'] > 0 else 1
    if price_diff <= 0.3:
        score += 30 * (1 - price_diff/0.3)
    elif price_diff <= 1.0:
        score += 15 * (1 - price_diff)
    
    # Property Type (25% or 20%)
    type_weight = 20 if has_preference else 25
    if target['type_clean'] == result['type_clean']:
        score += type_weight
    
    # Preference/Amenity (20% if selected)
    if has_preference:
        amenity_key = selected_amenity.lower().replace(' ', '_')
        if result.get(amenity_key, 0) == 1:
            score += 20
    
    # Area (15% or 10%)
    area_weight = 10 if has_preference else 15
    area_diff = abs(target['area'] - result['area']) / target['area'] if target['area'] > 0 else 1
    if area_diff <= 0.3:
        score += area_weight * (1 - area_diff/0.3)
    elif area_diff <= 0.5:
        score += (area_weight/2) * (1 - area_diff/0.5)
    
    # Bathrooms (15% or 10%)
    bath_weight = 10 if has_preference else 15
    if target['bathrooms'] == result['bathrooms']:
        score += bath_weight
    elif abs(target['bathrooms'] - result['bathrooms']) == 1:
        score += bath_weight/2
    
    # Bedrooms (15% or 10%)
    bed_weight = 10 if has_preference else 15
    if target['bedrooms'] == result['bedrooms']:
        score += bed_weight
    elif abs(target['bedrooms'] - result['bedrooms']) == 1:
        score += bed_weight/2
    
    return min(score, 100)

# --- ACCURACY EVALUATION ---
print("\nEvaluating model (City-First Strategy)...")

total_tests = 200
cities_tested = {}
matches_by_score = {'90-100': 0, '80-89': 0, '70-79': 0, '60-69': 0, '<60': 0}
test_amenities = ['pool', 'gym', 'parking', 'garden']

for i in np.random.randint(0, len(df_clean), total_tests):
    sample = df_clean.iloc[[i]]
    target_city = sample['city'].values[0]
    
    # Count properties per city
    if target_city not in cities_tested:
        cities_tested[target_city] = df_clean[df_clean['city'] == target_city].shape[0]
    
    # Filter by city FIRST
    city_properties = df_clean[df_clean['city'] == target_city]
    
    if len(city_properties) < 2:
        continue
    
    # Get KNN within city
    city_indices = city_properties.index.tolist()
    city_X = X_processed[city_indices]
    
    sample_vec = preprocessor.transform(sample)
    
    city_model = NearestNeighbors(n_neighbors=min(20, len(city_properties)), metric='cosine')
    city_model.fit(city_X)
    
    distances, indices = city_model.kneighbors(sample_vec)
    
    target_dict = sample.iloc[0].to_dict()
    test_amenity = np.random.choice(test_amenities + [None])
    
    for idx in indices[0][1:6]:  # Top 5
        global_idx = city_indices[idx]
        result = df_clean.iloc[global_idx]
        
        match_score = calculate_match_score(target_dict, result.to_dict(), test_amenity)
        
        if match_score >= 90:
            matches_by_score['90-100'] += 1
        elif match_score >= 80:
            matches_by_score['80-89'] += 1
        elif match_score >= 70:
            matches_by_score['70-79'] += 1
        elif match_score >= 60:
            matches_by_score['60-69'] += 1
        else:
            matches_by_score['<60'] += 1

print(f"\n{'='*60}")
print(f"MODEL ACCURACY REPORT (City-Filter Strategy)")
print(f"{'='*60}")
print(f"\n City Statistics:")
print(f"  • Cities tested: {len(cities_tested)}")
print(f"  • Avg properties/city: {np.mean(list(cities_tested.values())):.0f}")
print(f"  • Min properties/city: {min(cities_tested.values())}")
print(f"  • Max properties/city: {max(cities_tested.values())}")

print(f"\nMatch Score Distribution:")
total_matches = sum(matches_by_score.values())
for range_name, count in matches_by_score.items():
    pct = (count/total_matches)*100 if total_matches > 0 else 0
    print(f"  • {range_name}%: {count} ({pct:.1f}%)")

print(f"{'='*60}\n")


print("Saving model...")
artifacts = {
    'model': model,
    'preprocessor': preprocessor,
    'db_data': df_clean,
    'amenity_cols': amenity_cols,
    'type_weight': type_weight,
    'price_weight': price_weight,
    'area_weight': area_weight,
    'bed_bath_weight': bed_bath_weight
}

with open('recommender.pkl', 'wb') as f:
    pickle.dump(artifacts, f)

print("Model saved to 'recommender.pkl'")
print(f"\n Model Configuration:")
print(f"  • Total properties: {len(df_clean):,}")
print(f"  • Cities: {df_clean['city'].nunique()}")
print(f"  • Property types: {df_clean['type_clean'].nunique()}")
print(f"  • Strategy: City-First Filter")
print(f"\nScoring Weights:")
print(f"  NO Preference:")
print(f"    Budget(30%) + Type(25%) + Area(15%) + Bath(15%) + Bed(15%)")
print(f"  WITH Preference:")
print(f"    Budget(30%) + Type(20%) + Preference(20%) + Area(10%) + Bath(10%) + Bed(10%)\n")

Cleaning data...
Cleaned data: 11081 rows
Creating weighted features...
Features: 17 numeric, 3 categorical
Transforming features...
Transformed shape: (11081, 141)
Training KNN model...
Model trained!

Evaluating model (City-First Strategy)...

MODEL ACCURACY REPORT (City-Filter Strategy)

 City Statistics:
  • Cities tested: 7
  • Avg properties/city: 1572
  • Min properties/city: 106
  • Max properties/city: 8231

Match Score Distribution:
  • 90-100%: 45 (4.5%)
  • 80-89%: 63 (6.3%)
  • 70-79%: 182 (18.2%)
  • 60-69%: 240 (24.0%)
  • <60%: 470 (47.0%)

Saving model...
Model saved to 'recommender.pkl'

 Model Configuration:
  • Total properties: 11,081
  • Cities: 12
  • Property types: 8
  • Strategy: City-First Filter

Scoring Weights:
  NO Preference:
    Budget(30%) + Type(25%) + Area(15%) + Bath(15%) + Bed(15%)
  WITH Preference:
    Budget(30%) + Type(20%) + Preference(20%) + Area(10%) + Bath(10%) + Bed(10%)



In [22]:
%%writefile app.py
from flask import Flask, request, jsonify, render_template
import pandas as pd
import pickle
import numpy as np
from sklearn.neighbors import NearestNeighbors
import json

app = Flask(__name__)

# Load both models
RECOMMENDER_PATH = 'recommender.pkl'
ANALYTICS_PATH = 'analytics_model.pkl'

# Load Recommender
with open(RECOMMENDER_PATH, 'rb') as f:
    rec_artifacts = pickle.load(f)

rec_model = rec_artifacts['model']
rec_preprocessor = rec_artifacts['preprocessor']
rec_db_data = rec_artifacts['db_data']
rec_amenity_cols = rec_artifacts['amenity_cols']
type_weight = rec_artifacts.get('type_weight', 3)
price_weight = rec_artifacts.get('price_weight', 3)
area_weight = rec_artifacts.get('area_weight', 2)
bed_bath_weight = rec_artifacts.get('bed_bath_weight', 2)

# Load Analytics
with open(ANALYTICS_PATH, 'rb') as f:
    analytics_artifacts = pickle.load(f)

analytics_data = analytics_artifacts['analytics_data']
  
MAJOR_CITIES = [
    'Cairo', 'Alexandria', 'Giza', 'Red Sea', 'Suez', 
    'Damietta', 'Aswan', 'South Sinai', 'Matruh', 
    'Sokhna', 'North Coast'
]

print("Both models loaded successfully!")

# ========== LANDING PAGE ==========
@app.route('/')
def landing():
    return render_template('landing.html')

# ========== RECOMMENDER PAGES ==========
@app.route('/recommender')
def recommender_page():
    all_cities = rec_db_data['city'].dropna().unique()
    city_options = [c for c in MAJOR_CITIES if c in all_cities]
    
    types = sorted(rec_db_data['type_clean'].dropna().unique())
    amenities = [col.replace('_', ' ').title() for col in rec_amenity_cols]
    
    return render_template('recommender.html', 
                         cities=city_options,
                         types=types, 
                         amenities=amenities)

def calculate_match_score(target, result, selected_amenity=None):
    score = 0
    has_preference = bool(selected_amenity and selected_amenity.strip())
    
    # Budget (30%)
    price_diff = abs(target['price'] - result['price']) / target['price'] if target['price'] > 0 else 1
    if price_diff <= 0.3:
        score += 30 * (1 - price_diff/0.3)
    elif price_diff <= 1.0:
        score += 15 * (1 - price_diff)
    
    # Type (25% or 20%)
    tw = 20 if has_preference else 25
    if target['type_clean'] == result['type_clean']:
        score += tw
    
    # Preference (20%)
    if has_preference:
        amenity_key = selected_amenity.lower().replace(' ', '_')
        if result.get(amenity_key, 0) == 1:
            score += 20
    
    # Area (15% or 10%)
    aw = 10 if has_preference else 15
    area_diff = abs(target['area'] - result['area']) / target['area'] if target['area'] > 0 else 1
    if area_diff <= 0.3:
        score += aw * (1 - area_diff/0.3)
    elif area_diff <= 0.5:
        score += (aw/2) * (1 - area_diff/0.5)
    
    # Bath (15% or 10%)
    bw = 10 if has_preference else 15
    if target['bathrooms'] == result['bathrooms']:
        score += bw
    elif abs(target['bathrooms'] - result['bathrooms']) == 1:
        score += bw/2
    
    # Bed (15% or 10%)
    bdw = 10 if has_preference else 15
    if target['bedrooms'] == result['bedrooms']:
        score += bdw
    elif abs(target['bedrooms'] - result['bedrooms']) == 1:
        score += bdw/2
    
    return min(score, 100)

@app.route('/api/recommend', methods=['POST'])
def recommend():
    try:
        data = request.json
        
        input_data = {
            'area': float(data.get('area', 100)),
            'bedrooms': int(data.get('bedrooms', 3)),
            'bathrooms': int(data.get('bathrooms', 2)),
            'price': float(data.get('price', 5000000)),
            'city': data.get('city', 'Cairo'),
            'type_clean': data.get('property_type', 'Apartment'),
            'text_features': ''
        }

        selected_pref = data.get('preferences', '').strip()
        selected_city = input_data['city']
        
        city_properties = rec_db_data[rec_db_data['city'] == selected_city].copy()
        
        if len(city_properties) == 0:
            return jsonify({'error': 'No properties found in this city'}), 404
        
        for i in range(type_weight):
            input_data[f'type_{i}'] = input_data['type_clean']
        for i in range(price_weight):
            input_data[f'price_{i}'] = input_data['price']
        for i in range(area_weight):
            input_data[f'area_{i}'] = input_data['area']
        for i in range(bed_bath_weight):
            input_data[f'bed_{i}'] = input_data['bedrooms']
            input_data[f'bath_{i}'] = input_data['bathrooms']
        
        for col in rec_amenity_cols:
            input_data[col] = 0
        if selected_pref:
            col_name = selected_pref.lower().replace(' ', '_')
            if col_name in rec_amenity_cols:
                input_data[col_name] = 1
        
        q = pd.DataFrame([input_data])
        input_vec = rec_preprocessor.transform(q)
        
        filtered_indices = city_properties.index.tolist()
        filtered_X = rec_preprocessor.transform(city_properties)
        
        n_neighbors = min(len(city_properties), 50)
        filtered_model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
        filtered_model.fit(filtered_X)
        
        distances, indices = filtered_model.kneighbors(input_vec)
        
        candidates = []
        for local_idx in indices[0]:
            global_idx = filtered_indices[local_idx]
            prop = rec_db_data.iloc[global_idx]
            match_score = calculate_match_score(input_data, prop.to_dict(), selected_pref)
            candidates.append({'idx': global_idx, 'score': match_score})
        
        candidates = sorted(candidates, key=lambda x: x['score'], reverse=True)
        num_results = min(5, len(candidates))
        candidates = candidates[:num_results]
        
        results = []
        for c in candidates:
            prop = rec_db_data.iloc[c['idx']].to_dict()
            prop['score'] = int(c['score'])
            prop['type_match'] = prop['type_clean'] == input_data['type_clean']
            
            if selected_pref:
                amenity_key = selected_pref.lower().replace(' ', '_')
                prop['has_amenity'] = bool(prop.get(amenity_key, 0))
            else:
                prop['has_amenity'] = None
            
            for k, v in prop.items():
                if pd.isna(v): prop[k] = None
                
            results.append({
                'title': str(prop.get('title', 'Property')),
                'region': str(prop.get('region', 'N/A')),
                'city': str(prop.get('city', 'N/A')),
                'price': float(prop.get('price', 0)),
                'area': float(prop.get('area', 0)),
                'bedrooms': int(prop.get('bedrooms', 0)),
                'bathrooms': int(prop.get('bathrooms', 0)),
                'type_clean': str(prop.get('type_clean', 'Property')),
                'score': int(prop['score']),
                'type_match': bool(prop['type_match']),
                'link': str(prop.get('link', ''))
            })
        
        return jsonify(results)
    
    except Exception as e:
        import traceback
        print(traceback.format_exc())
        return jsonify({'error': str(e)}), 500

# ========== ANALYTICS PAGES ==========
@app.route('/analytics')
def analytics_page():
    available_cities = [c for c in MAJOR_CITIES if c in analytics_data]
    
    city_region_map = {}
    for city in available_cities:
        regions = sorted(list(analytics_data[city]['regions'].keys()))
        city_region_map[city] = regions
        
    return render_template('analytics.html', 
                         cities=available_cities, 
                         city_region_map=city_region_map)

@app.route('/api/analytics', methods=['POST'])
def get_analytics():
    try:
        data = request.json
        city = data.get('city')
        region = data.get('region', None)
        
        if city not in analytics_data:
            return jsonify({'error': 'City not found'}), 404
        
        city_info = analytics_data[city]
        
        if region and region != "" and region in city_info['regions']:
            region_info = city_info['regions'][region]
            return jsonify({
                'type': 'region',
                'city': city,
                'region': region,
                'total_properties': region_info['total_properties'],
                'avg_price': f"{region_info['avg_price']:,.0f} EGP",
                'median_price': f"{region_info['median_price']:,.0f} EGP",
                'avg_area': f"{region_info['avg_area']:.0f} sqm",
                'top_property_type': region_info['top_property_type'],
                'property_distribution': region_info['property_type_distribution'],
                'status': region_info['status']
            })
        else:
            hotspots = [r for r, info in city_info['regions'].items() if info['status'] == 'Hotspot']
            coldspots = [r for r, info in city_info['regions'].items() if info['status'] == 'Coldspot']
            
            return jsonify({
                'type': 'city',
                'city': city,
                'total_properties': city_info['total_properties'],
                'avg_price': f"{city_info['avg_price']:,.0f} EGP",
                'median_price': f"{city_info['median_price']:,.0f} EGP",
                'price_range': f"{city_info['min_price']:,.0f} - {city_info['max_price']:,.0f} EGP",
                'avg_area': f"{city_info['avg_area']:.0f} sqm",
                'top_property_type': city_info['top_property_type'],
                'property_distribution': city_info['property_type_distribution'],
                'hotspots': hotspots,
                'coldspots': coldspots
            })
    
    except Exception as e:
        import traceback
        print(traceback.format_exc())
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    print("Unified App running on http://0.0.0.0:5000")
    print("Models: Recommender + Analytics")
    app.run(host='0.0.0.0', port=5000, debug=True)

Overwriting app.py


In [23]:
!mkdir -p templates

In [24]:
%%writefile templates/landing.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Real Estate AI Platform</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <link href="https://fonts.googleapis.com/css2?family=Playfair+Display:wght@700&family=Poppins:wght@300;400;500;600;700&display=swap" rel="stylesheet">
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }
        
        body {
            background: linear-gradient(135deg, #CABA9C 0%, #8A6240 100%);
            color: #1C2820;
            min-height: 100vh;
            font-family: 'Poppins', sans-serif;
            padding: 40px 20px;
            position: relative;
            overflow-x: hidden;
        }
        
        /* Animated Background Pattern */
        body::before {
            content: '';
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background-image: 
                radial-gradient(circle at 20% 50%, rgba(76, 100, 68, 0.15) 0%, transparent 50%),
                radial-gradient(circle at 80% 80%, rgba(77, 45, 24, 0.15) 0%, transparent 50%);
            animation: float 20s ease-in-out infinite;
            pointer-events: none;
            z-index: 0;
        }
        
        @keyframes float {
            0%, 100% { transform: translateY(0px); }
            50% { transform: translateY(-20px); }
        }
        
        .main-container {
            max-width: 1200px;
            margin: 0 auto;
            position: relative;
            z-index: 1;
        }
        
        .header-section {
            text-align: center;
            margin-bottom: 60px;
            animation: fadeInDown 1s ease;
        }
        
        @keyframes fadeInDown {
            from {
                opacity: 0;
                transform: translateY(-30px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }
        
        .main-title {
            color: #1C2820;
            font-size: 3.5rem;
            font-weight: 700;
            margin-bottom: 20px;
            font-family: 'Playfair Display', serif;
            background: linear-gradient(135deg, #1C2820 0%, #4C6444 50%, #8A6240 100%);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            background-clip: text;
        }
        
        .subtitle {
            color: #4D2D18;
            font-size: 1.3rem;
            font-weight: 300;
            letter-spacing: 1px;
            animation: fadeIn 1.5s ease;
        }
        
        @keyframes fadeIn {
            from { opacity: 0; }
            to { opacity: 1; }
        }
        
        .stats-bar {
            display: flex;
            justify-content: space-around;
            margin: 40px 0;
            padding: 30px;
            background: rgba(255, 255, 255, 0.3);
            backdrop-filter: blur(10px);
            border-radius: 20px;
            border: 2px solid rgba(76, 100, 68, 0.3);
            animation: slideUp 1s ease;
        }
        
        @keyframes slideUp {
            from {
                opacity: 0;
                transform: translateY(30px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }
        
        .stat-item {
            text-align: center;
        }
        
        .stat-number {
            font-size: 2.5rem;
            font-weight: 700;
            color: #4C6444;
            font-family: 'Playfair Display', serif;
        }
        
        .stat-label {
            font-size: 0.9rem;
            color: #4D2D18;
            margin-top: 5px;
            text-transform: uppercase;
            letter-spacing: 1px;
        }
        
        .cards-container {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(450px, 1fr));
            gap: 40px;
            margin-top: 40px;
        }
        
        .feature-card {
            background: rgba(255, 255, 255, 0.4);
            backdrop-filter: blur(10px);
            border-radius: 25px;
            padding: 45px 40px;
            box-shadow: 
                0 15px 35px rgba(28, 40, 32, 0.15),
                0 5px 15px rgba(28, 40, 32, 0.08);
            transition: all 0.5s cubic-bezier(0.4, 0, 0.2, 1);
            border: 2px solid rgba(76, 100, 68, 0.3);
            position: relative;
            overflow: hidden;
            animation: fadeInUp 1s ease;
            animation-fill-mode: both;
        }
        
        .feature-card:nth-child(1) {
            animation-delay: 0.2s;
        }
        
        .feature-card:nth-child(2) {
            animation-delay: 0.4s;
        }
        
        @keyframes fadeInUp {
            from {
                opacity: 0;
                transform: translateY(50px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }
        
        .feature-card::before {
            content: '';
            position: absolute;
            top: 0;
            left: -100%;
            width: 100%;
            height: 4px;
            background: linear-gradient(90deg, transparent, #4C6444, transparent);
            transition: left 0.5s ease;
        }
        
        .feature-card:hover::before {
            left: 100%;
        }
        
        .feature-card::after {
            content: '';
            position: absolute;
            top: 0;
            right: 0;
            bottom: 0;
            left: 0;
            background: linear-gradient(135deg, rgba(76, 100, 68, 0.05) 0%, rgba(138, 98, 64, 0.05) 100%);
            opacity: 0;
            transition: opacity 0.5s ease;
            pointer-events: none;
        }
        
        .feature-card:hover {
            transform: translateY(-12px) scale(1.02);
            box-shadow: 
                0 25px 50px rgba(28, 40, 32, 0.25),
                0 10px 25px rgba(28, 40, 32, 0.18);
            border-color: #4C6444;
        }
        
        .feature-card:hover::after {
            opacity: 1;
        }
        
        .card-title {
            color: #1C2820;
            font-size: 2rem;
            font-weight: 700;
            margin-bottom: 20px;
            font-family: 'Playfair Display', serif;
        }
        
        .card-description {
            color: #4D2D18;
            font-size: 1.05rem;
            line-height: 1.7;
            margin-bottom: 25px;
        }
        
        .features-list {
            list-style: none;
            margin-bottom: 30px;
            padding: 0;
        }
        
        .features-list li {
            color: #1C2820;
            padding: 14px 0;
            padding-left: 35px;
            position: relative;
            font-size: 1rem;
            border-bottom: 1px solid rgba(76, 100, 68, 0.3);
            transition: all 0.3s ease;
            opacity: 0;
            animation: slideInLeft 0.5s ease forwards;
        }
        
        .features-list li:nth-child(1) { animation-delay: 0.6s; }
        .features-list li:nth-child(2) { animation-delay: 0.7s; }
        .features-list li:nth-child(3) { animation-delay: 0.8s; }
        .features-list li:nth-child(4) { animation-delay: 0.9s; }
        
        @keyframes slideInLeft {
            from {
                opacity: 0;
                transform: translateX(-20px);
            }
            to {
                opacity: 1;
                transform: translateX(0);
            }
        }
        
        .features-list li:hover {
            padding-left: 40px;
            color: #4D2D18;
        }
        
        .features-list li:last-child {
            border-bottom: none;
        }
        
        .features-list li::before {
            content: '✓';
            position: absolute;
            left: 0;
            color: #4C6444;
            font-weight: bold;
            font-size: 1.3rem;
            transition: all 0.3s ease;
        }
        
        .features-list li:hover::before {
            transform: scale(1.3) rotate(360deg);
        }
        
        .cta-button {
            width: 100%;
            padding: 18px 32px;
            border: none;
            border-radius: 15px;
            font-size: 1.1rem;
            font-weight: 600;
            cursor: pointer;
            transition: all 0.4s cubic-bezier(0.4, 0, 0.2, 1);
            text-transform: uppercase;
            letter-spacing: 1.5px;
            position: relative;
            overflow: hidden;
            z-index: 1;
        }
        
        .cta-button::before {
            content: '';
            position: absolute;
            top: 50%;
            left: 50%;
            width: 0;
            height: 0;
            border-radius: 50%;
            background: rgba(255, 255, 255, 0.3);
            transform: translate(-50%, -50%);
            transition: width 0.6s, height 0.6s;
            z-index: -1;
        }
        
        .cta-button:hover::before {
            width: 300px;
            height: 300px;
        }
        
        .btn-recommender {
            background: linear-gradient(135deg, #4C6444 0%, #8A6240 100%);
            color: white;
            box-shadow: 0 8px 20px rgba(76, 100, 68, 0.4);
        }
        
        .btn-recommender:hover {
            background: linear-gradient(135deg, #1C2820 0%, #4C6444 100%);
            box-shadow: 0 12px 30px rgba(76, 100, 68, 0.5);
            transform: translateY(-3px);
        }
        
        .btn-analytics {
            background: linear-gradient(135deg, #8A6240 0%, #4D2D18 100%);
            color: white;
            box-shadow: 0 8px 20px rgba(138, 98, 64, 0.4);
        }
        
        .btn-analytics:hover {
            background: linear-gradient(135deg, #4C6444 0%, #8A6240 100%);
            box-shadow: 0 12px 30px rgba(138, 98, 64, 0.5);
            transform: translateY(-3px);
        }
        
        .arrow {
            display: inline-block;
            margin-left: 10px;
            transition: transform 0.3s ease;
        }
        
        .cta-button:hover .arrow {
            transform: translateX(8px);
        }
        
        @media (max-width: 992px) {
            .cards-container {
                grid-template-columns: 1fr;
            }
            
            .main-title {
                font-size: 2.5rem;
            }
            
            .subtitle {
                font-size: 1.1rem;
            }
            
            .stats-bar {
                flex-direction: column;
                gap: 20px;
            }
        }
        
        @media (max-width: 576px) {
            .main-title {
                font-size: 2rem;
            }
            
            .feature-card {
                padding: 35px 25px;
            }
            
            .card-title {
                font-size: 1.5rem;
            }
        }
    </style>
</head>
<body>
    <div class="main-container">
        <div class="header-section">
            <h1 class="main-title">🏠 Real Estate AI Platform</h1>
            <p class="subtitle">Professional Intelligence for Smart Property Decisions</p>
        </div>
        
        <div class="stats-bar">
            <div class="stat-item">
                <div class="stat-number" data-target="1500">0</div>
                <div class="stat-label">Properties</div>
            </div>
            <div class="stat-item">
                <div class="stat-number" data-target="25">0</div>
                <div class="stat-label">Cities</div>
            </div>
            <div class="stat-item">
                <div class="stat-number" data-target="500">0</div>
                <div class="stat-label">Happy Clients</div>
            </div>
        </div>
        
        <div class="cards-container">
            <div class="feature-card">
                <h2 class="card-title">Property Recommender</h2>
                <p class="card-description">
                    Discover your ideal property with our advanced AI-powered matching system tailored to your specific requirements.
                </p>
                <ul class="features-list">
                    <li>Intelligent AI Match Scoring</li>
                    <li>Advanced Budget & Amenity Filtering</li>
                    <li>Top 5 Personalized Recommendations</li>
                    <li>Real-time Property Availability</li>
                </ul>
                <button class="cta-button btn-recommender" onclick="window.location.href='/recommender'">
                    Start Your Search <span class="arrow">→</span>
                </button>
            </div>
            
            <div class="feature-card">
                <h2 class="card-title">Market Analytics</h2>
                <p class="card-description">
                    Gain comprehensive insights into real estate market trends, pricing dynamics, and regional opportunities.
                </p>
                <ul class="features-list">
                    <li>Regional Hotspots & Coldspots Analysis</li>
                    <li>Detailed Price & Area Statistics</li>
                    <li>Dynamic Market Trend Insights</li>
                    <li>Investment Opportunity Indicators</li>
                </ul>
                <button class="cta-button btn-analytics" onclick="window.location.href='/analytics'">
                    View Analytics <span class="arrow">→</span>
                </button>
            </div>
        </div>
    </div>
    
    <script>
        // Animated Counter
        function animateCounter(element) {
            const target = parseInt(element.getAttribute('data-target'));
            const duration = 2000;
            const step = target / (duration / 16);
            let current = 0;
            
            const timer = setInterval(() => {
                current += step;
                if (current >= target) {
                    element.textContent = target + '+';
                    clearInterval(timer);
                } else {
                    element.textContent = Math.floor(current);
                }
            }, 16);
        }
        
        // Trigger counter animation on page load
        window.addEventListener('load', () => {
            document.querySelectorAll('.stat-number').forEach(animateCounter);
        });
    </script>
</body>
</html>

Overwriting templates/landing.html


In [25]:
%%writefile templates/recommender.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>AI Property Matcher</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <link href="https://fonts.googleapis.com/css2?family=Playfair+Display:wght@700&family=Poppins:wght@300;400;500;600;700&display=swap" rel="stylesheet">

    <style>

        /* ==============================
           GLOBAL PAGE COLORS
        ===============================*/
        body { 
            background: #1C2820;           /* Dark Green (same as analytics) */
            color: #E8DCC2;               /* Light Beige */
            font-family: 'Poppins', sans-serif;
            min-height: 100vh;
            padding: 40px 0;
        }

        /* ==============================
           CARD STYLE
        ===============================*/
        .card { 
            background: #2A3A2E;          /* Same card color as analytics */
            border: 2px solid #4C6444;    /* Green border */
            color: #E8DCC2 !important;
            box-shadow: 0 10px 30px rgba(0, 0, 0, 0.25);
            border-radius: 20px;
            animation: fadeIn 0.6s ease;
        }

        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(20px); }
            to { opacity: 1; transform: translateY(0); }
        }

        /* ==============================
           FORM INPUTS
        ===============================*/
        .form-control, .form-select { 
            background-color: #3c4c41;
            border: 2px solid #4C6444;
            color: #E8DCC2 !important;
            font-weight: 500;
            transition: all 0.3s ease;
        }
        
        .form-control:focus, .form-select:focus { 
            background-color: #2A3A2E;
            color: #E8DCC2 !important; 
            border-color: #8A6240; 
            box-shadow: 0 0 0 0.25rem rgba(138,98,64,0.25);
            transform: translateY(-2px);
        }

        .form-label {
            color: #CABA9C !important;
            font-weight: 600;
        }

        /* ==============================
           PRIMARY BUTTON
        ===============================*/
        .btn-primary { 
            background: #4C6444;
            border: none; 
            font-weight: 600;
            padding: 14px;
            color: #fff;
            box-shadow: 0 4px 15px rgba(76, 100, 68, 0.3);
            transition: all 0.4s ease;
            position: relative;
            overflow: hidden;
        }

        .btn-primary::before {
            content: '';
            position: absolute;
            top: 50%;
            left: 50%;
            width: 0;
            height: 0;
            border-radius: 50%;
            background: rgba(255, 255, 255, 0.25);
            transform: translate(-50%, -50%);
            transition: width 0.6s, height 0.6s;
        }

        .btn-primary:hover::before {
            width: 400px;
            height: 400px;
        }

        .btn-primary:hover { 
            background: #1C2820;
            transform: translateY(-3px);
            box-shadow: 0 6px 25px rgba(0,0,0,0.35);
        }

        .btn-outline-light { 
            border: 2px solid #8A6240;
            color: #E8DCC2;
            background-color: transparent;
            font-weight: 600;
            transition: all 0.3s ease;
        }

        .btn-outline-light:hover { 
            background-color: #8A6240; 
            color: #fff;
            transform: translateX(-5px);
        }

        /* ==============================
           PAGE TITLE
        ===============================*/
        h1 {
            color: #E8DCC2 !important;
            font-weight: 700;
            font-family: 'Playfair Display', serif;
            text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.45);
            animation: fadeInDown 0.8s ease;
        }

        @keyframes fadeInDown {
            from { opacity: 0; transform: translateY(-30px); }
            to { opacity: 1; transform: translateY(0); }
        }

        /* ==============================
           LOADING SPINNER
        ===============================*/
        .loading-spinner {
            display: inline-block;
            width: 50px;
            height: 50px;
            border: 5px solid rgba(255,255,255,0.15);
            border-top-color: #CABA9C;
            border-radius: 50%;
            animation: spin 1s linear infinite;
        }
        
        @keyframes spin {
            to { transform: rotate(360deg); }
        }

        .loading-text {
            color: #CABA9C !important;
            font-weight: 600;
            margin-top: 15px;
            animation: pulse 1.5s ease infinite;
        }

        @keyframes pulse {
            0%, 100% { opacity: 1; }
            50% { opacity: 0.5; }
        }

        /* ==============================
           RECOMMENDATION CARDS
        ===============================*/
        .result-card {
            background: #2A3A2E;
            border: 2px solid #4C6444 !important;
            border-left: 6px solid #8A6240 !important;
            color: #E8DCC2 !important;
            border-radius: 15px;
            transition: all 0.4s cubic-bezier(0.4, 0, 0.2, 1);
            animation: slideIn 0.5s ease both;
            position: relative;
            overflow: hidden;
        }
        
        /* SHINE EFFECT */
        .result-card::before {
            content: '';
            position: absolute;
            top: 0;
            left: -100%;
            width: 100%;
            height: 100%;
            background: linear-gradient(90deg, transparent, rgba(255,255,255,0.08), transparent);
            transition: left 0.5s ease;
        }
        
        .result-card:hover::before {
            left: 100%;
        }
        
        @keyframes slideIn {
            from { opacity: 0; transform: translateX(-30px); }
            to { opacity: 1; transform: translateX(0); }
        }

        .result-card:hover {
            transform: translateX(14px) translateY(-6px);
            box-shadow: 0 15px 40px rgba(0,0,0,0.35);
            border-left-width: 10px !important;
        }

        .result-card h5 {
            color: #E8DCC2 !important;
            font-weight: 600;
            font-family: 'Playfair Display', serif;
        }

        .result-card small {
            color: #CABA9C !important;
            font-weight: 500;
        }

        /* MATCH BADGES */
        .badge {
            padding: 10px 16px;
            font-weight: 600;
            border-radius: 10px;
            animation: bounceIn 0.6s ease;
        }

        @keyframes bounceIn {
            0% { transform: scale(0); }
            50% { transform: scale(1.1); }
            100% { transform: scale(1); }
        }

        .bg-success { background: #4C6444 !important; color: white !important; }
        .bg-warning { background: #8A6240 !important; color: white !important; }

        /* PRICE */
        .price-text {
            color: #CABA9C !important;
            font-weight: 700;
            font-size: 1.2rem;
            font-family: 'Playfair Display', serif;
        }

        .property-details {
            color: #CABA9C !important;
            font-weight: 500;
        }

        /* VIEW LISTING BUTTON */
        .btn-outline-info {
            border: 2px solid #8A6240;
            color: #8A6240;
            background-color: transparent;
            font-weight: 600;
            transition: all 0.3s ease;
        }
        
        .btn-outline-info:hover {
            background-color: #8A6240;
            color: #fff;
            transform: scale(1.05);
        }

        /* ALERTS */
        .alert-danger {
            background-color: #4c0000;
            border: 2px solid #c62828;
            color: #ffb3b3;
            border-radius: 12px;
        }
        
        .alert-warning {
            background-color: #3d2a00;
            border: 2px solid #ff8f00;
            color: #ffcc80;
            border-radius: 12px;
        }

    </style>
</head>

<body>
<div class="container py-5">

    <a href="/" class="btn btn-outline-light mb-3">← Back to Home</a>

    <h1 class="text-center mb-4 fw-bold">🎯 AI Property Matcher</h1>

    <div class="row justify-content-center">

        <div class="col-md-5">
            <div class="card p-4 shadow-lg mb-4">

                <form id="form">
                    <div class="mb-3">
                        <label class="form-label">Budget (EGP)</label>
                        <input type="number" id="price" class="form-control" value="5000000">
                    </div>

                    <div class="mb-3">
                        <label class="form-label">Area (sqm)</label>
                        <input type="number" id="area" class="form-control" value="150">
                    </div>

                    <div class="mb-3">
                        <label class="form-label">City</label>
                        <select id="city" class="form-select">
                            {% for city in cities %}
                            <option value="{{ city }}">{{ city }}</option>
                            {% endfor %}
                        </select>
                    </div>

                    <div class="mb-3">
                        <label class="form-label">Property Type</label>
                        <select id="type" class="form-select">
                            {% for t in types %}
                            <option value="{{ t }}">{{ t }}</option>
                            {% endfor %}
                        </select>
                    </div>

                    <div class="row mb-3">
                        <div class="col">
                            <label class="form-label">Beds</label>
                            <input type="number" id="bedrooms" class="form-control" value="3">
                        </div>
                        <div class="col">
                            <label class="form-label">Baths</label>
                            <input type="number" id="bathrooms" class="form-control" value="2">
                        </div>
                    </div>

                    <div class="mb-4">
                        <label class="form-label">Must Have</label>
                        <select id="keywords" class="form-select">
                            <option value="">No Preference</option>
                            {% for a in amenities %}
                            <option value="{{ a }}">{{ a }}</option>
                            {% endfor %}
                        </select>
                    </div>

                    <button type="submit" class="btn btn-primary w-100 py-2">Find Matches</button>
                </form>

            </div>
        </div>

        <div class="col-md-7">
            <div id="results"></div>
        </div>

    </div>
</div>

<script>
document.getElementById('form').addEventListener('submit', async (e) => {
    e.preventDefault();
    const resDiv = document.getElementById('results');

    resDiv.innerHTML = `
        <div class="text-center">
            <div class="loading-spinner"></div>
            <div class="loading-text">🔍 Searching best properties...</div>
        </div>
    `;
    
    try {
        const res = await fetch('/api/recommend', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({
                price: document.getElementById('price').value,
                area: document.getElementById('area').value,
                city: document.getElementById('city').value,
                property_type: document.getElementById('type').value,
                bedrooms: document.getElementById('bedrooms').value,
                bathrooms: document.getElementById('bathrooms').value,
                preferences: document.getElementById('keywords').value
            })
        });
        
        const data = await res.json();
        
        setTimeout(() => {
            resDiv.innerHTML = '';
            
            if (data.error) {
                 resDiv.innerHTML = `<div class="alert alert-danger">Error: ${data.error}</div>`;
                 return;
            }
            
            if (data.length === 0) {
                 resDiv.innerHTML = '<div class="alert alert-warning">No matches found in this city.</div>';
                 return;
            }

            data.forEach(item => {
                let badgeColor = item.type_match ? 'bg-success' : 'bg-warning';
                let matchText = item.type_match ? item.score + '% Match' : 'Possible Match (' + item.type_clean + ')';
                
                resDiv.innerHTML += `
                    <div class="card result-card p-3 mb-3 shadow-sm">
                        <div class="d-flex justify-content-between">
                            <div>
                                <h5 class="mb-1">${item.title || 'Property'}</h5>
                                <small>${item.region || 'N/A'}, ${item.city}</small>
                            </div>
                            <div class="text-end">
                                <span class="badge ${badgeColor} mb-1">${matchText}</span>
                                <div class="price-text">${parseInt(item.price).toLocaleString()} EGP</div>
                            </div>
                        </div>
                        <div class="mt-2 d-flex justify-content-between align-items-center">
                            <small class="property-details">${item.type_clean} • ${item.bedrooms} Bed • ${item.bathrooms} Bath • ${item.area} sqm</small>
                            ${item.link ? `<a href="${item.link}" target="_blank" class="btn btn-sm btn-outline-info">View Listing</a>` : ''}
                        </div>
                    </div>`;
            });
        }, 300);
        
    } catch (err) {
        resDiv.innerHTML = `<div class="alert alert-danger">Error: ${err.message}</div>`;
    }
});
</script>

</body>
</html>


Overwriting templates/recommender.html


In [26]:
%%writefile templates/analytics.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Market Analytics</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <link href="https://fonts.googleapis.com/css2?family=Playfair+Display:wght@700&family=Poppins:wght@300;400;500;600;700&display=swap" rel="stylesheet">
    <style>
        body { 
            background: #1C2820;
            color: #CABA9C; 
            font-family: 'Poppins', sans-serif;
            min-height: 100vh;
            padding: 40px 0;
        }
        
        .card { 
            background: #2a3a2e;
            border: 2px solid #4C6444;
            color: #CABA9C;
            box-shadow: 0 15px 40px rgba(0, 0, 0, 0.5);
            border-radius: 20px;
            animation: fadeIn 0.6s ease;
        }
        
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(30px); }
            to { opacity: 1; transform: translateY(0); }
        }
        
        .form-control, .form-select { 
            background-color: #3a4a3e; 
            border: 2px solid #4C6444; 
            color: #CABA9C !important;
            font-weight: 500;
            transition: all 0.3s ease;
        }
        
        .form-control:focus, .form-select:focus { 
            background-color: #4a5a4e; 
            color: #fff !important; 
            border-color: #8A6240; 
            box-shadow: 0 0 0 0.3rem rgba(138, 98, 64, 0.2);
            transform: translateY(-2px);
        }
        
        .form-label {
            color: #CABA9C !important;
            font-weight: 600;
            margin-bottom: 8px;
        }
        
        .btn-success { 
            background: #4C6444;
            border: none; 
            font-weight: 600;
            padding: 14px;
            box-shadow: 0 8px 25px rgba(0, 0, 0, 0.4);
            transition: all 0.4s ease;
            position: relative;
            overflow: hidden;
        }
        
        .btn-success::before {
            content: '';
            position: absolute;
            top: 50%;
            left: 50%;
            width: 0;
            height: 0;
            border-radius: 50%;
            background: rgba(255, 255, 255, 0.2);
            transform: translate(-50%, -50%);
            transition: width 0.6s, height 0.6s;
        }
        
        .btn-success:hover::before {
            width: 400px;
            height: 400px;
        }
        
        .btn-success:hover { 
            background: #8A6240;
            box-shadow: 0 10px 35px rgba(0, 0, 0, 0.6);
            transform: translateY(-3px);
        }
        
        .btn-outline-light { 
            border: 2px solid #4C6444; 
            color: #CABA9C;
            background-color: #2a3a2e;
            font-weight: 600;
            transition: all 0.3s ease;
        }
        
        .btn-outline-light:hover { 
            background-color: #4C6444; 
            color: #fff;
            border-color: #4C6444;
            transform: translateX(-5px);
        }
        
        .stat-card {
            background: #3a4a3e;
            padding: 30px;
            border-radius: 15px;
            margin: 15px 0;
            border: 2px solid #4C6444;
            box-shadow: 0 8px 25px rgba(0, 0, 0, 0.4);
            transition: all 0.4s ease;
            animation: slideIn 0.5s ease;
            animation-fill-mode: both;
        }
        
        @keyframes slideIn {
            from {
                opacity: 0;
                transform: translateX(-30px);
            }
            to {
                opacity: 1;
                transform: translateX(0);
            }
        }
        
        .stat-card:nth-child(1) { animation-delay: 0.1s; }
        .stat-card:nth-child(2) { animation-delay: 0.2s; }
        .stat-card:nth-child(3) { animation-delay: 0.3s; }
        
        .stat-card:hover {
            transform: translateX(10px) scale(1.02);
            box-shadow: 0 12px 35px rgba(0, 0, 0, 0.5);
            border-color: #8A6240;
            background: #4a5a4e;
        }
        
        .stat-card h5 {
            color: #CABA9C !important;
            font-weight: 600;
            margin-bottom: 15px;
        }
        
        .stat-card h2 {
            font-weight: 700;
            margin-bottom: 10px;
            font-family: 'Playfair Display', serif;
        }
        
        .stat-card small {
            color: #a09080 !important;
            font-weight: 500;
        }
        
        .hotspot { 
            color: #ff6b6b !important; 
            font-weight: bold;
            text-shadow: 0 0 10px rgba(255, 107, 107, 0.6);
            animation: glow 2s ease-in-out infinite;
        }
        
        @keyframes glow {
            0%, 100% { text-shadow: 0 0 10px rgba(255, 107, 107, 0.6); }
            50% { text-shadow: 0 0 20px rgba(255, 107, 107, 0.9); }
        }
        
        .coldspot { 
            color: #64b5f6 !important; 
            font-weight: bold;
            text-shadow: 0 0 10px rgba(100, 181, 246, 0.6);
            animation: glowBlue 2s ease-in-out infinite;
        }
        
        @keyframes glowBlue {
            0%, 100% { text-shadow: 0 0 10px rgba(100, 181, 246, 0.6); }
            50% { text-shadow: 0 0 20px rgba(100, 181, 246, 0.9); }
        }
        
        h1 { 
            color: #CABA9C !important;
            font-weight: 700;
            font-family: 'Playfair Display', serif;
            animation: fadeInDown 0.8s ease;
        }
        
        @keyframes fadeInDown {
            from { opacity: 0; transform: translateY(-30px); }
            to { opacity: 1; transform: translateY(0); }
        }
        
        h3 {
            color: #CABA9C !important;
            font-weight: 700;
            font-family: 'Playfair Display', serif;
        }
        
        h4 {
            font-weight: 600;
        }
        
        h5 {
            color: #CABA9C !important;
            font-weight: 600;
        }
        
        ul {
            color: #CABA9C !important;
        }
        
        ul li {
            margin: 10px 0;
            font-weight: 500;
            padding-left: 25px;
            position: relative;
        }
        
        ul li::before {
            content: '▸';
            position: absolute;
            left: 0;
            color: #8A6240;
            font-size: 1.2rem;
        }
        
        hr {
            border-color: #4C6444 !important;
            opacity: 0.5;
        }
        
        /* Loading State */
        .loading-spinner {
            display: inline-block;
            width: 50px;
            height: 50px;
            border: 5px solid rgba(202, 186, 156, 0.2);
            border-top-color: #CABA9C;
            border-radius: 50%;
            animation: spin 1s linear infinite;
        }
        
        @keyframes spin {
            to { transform: rotate(360deg); }
        }
        
        .text-info {
            color: #CABA9C !important;
            font-weight: 600;
            animation: pulse 1.5s ease infinite;
        }
        
        @keyframes pulse {
            0%, 100% { opacity: 1; }
            50% { opacity: 0.6; }
        }
        
        .alert-danger {
            background-color: #3d1f1f;
            border: 2px solid #c62828;
            color: #ff8a80;
            border-radius: 12px;
        }
        
        /* Price, Area, Type colors */
        .price-color {
            color: #8A6240 !important;
        }
        
        .area-color {
            color: #4C6444 !important;
        }
        
        .type-color {
            color: #CABA9C !important;
        }
        
        /* Results animation */
        .results-container {
            animation: fadeInUp 0.6s ease;
        }
        
        @keyframes fadeInUp {
            from {
                opacity: 0;
                transform: translateY(30px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }
        
        /* Overview header */
        .overview-header {
            display: flex;
            align-items: center;
            gap: 15px;
            margin-bottom: 20px;
        }
        
        .overview-icon {
            font-size: 2.5rem;
            animation: bounce 2s ease-in-out infinite;
        }
        
        @keyframes bounce {
            0%, 100% { transform: translateY(0); }
            50% { transform: translateY(-10px); }
        }
        
        /* Scrollbar */
        ::-webkit-scrollbar {
            width: 10px;
        }
        
        ::-webkit-scrollbar-track {
            background: #2a3a2e;
        }
        
        ::-webkit-scrollbar-thumb {
            background: #4C6444;
            border-radius: 5px;
        }
        
        ::-webkit-scrollbar-thumb:hover {
            background: #8A6240;
        }
    </style>
</head>
<body>
<div class="container py-5">
    <a href="/" class="btn btn-outline-light mb-3">← Back to Home</a>
    <h1 class="text-center mb-4 fw-bold">📊 Market Analytics</h1>
    
    <div class="row justify-content-center">
        <div class="col-md-5">
            <div class="card p-4 shadow-lg mb-4">
                <form id="analyticsForm">
                    <div class="mb-3">
                        <label class="form-label">City</label>
                        <select id="city" class="form-select">
                            <option value="" disabled selected>Select a city...</option>
                            {% for city in cities %}
                            <option value="{{ city }}">{{ city }}</option>
                            {% endfor %}
                        </select>
                    </div>
                    
                    <div class="mb-3" id="regionGroup">
                        <label class="form-label">Region (Optional)</label>
                        <select id="region" class="form-select" disabled>
                            <option value="">-- City Overview --</option>
                        </select>
                    </div>
                    
                    <button type="submit" class="btn btn-success w-100 py-2">Get Analytics</button>
                </form>
            </div>
        </div>
        
        <div class="col-md-7">
            <div id="results"></div>
        </div>
    </div>
</div>

<script>
    const cityRegionMap = {{ city_region_map | tojson }};
    const citySelect = document.getElementById('city');
    const regionSelect = document.getElementById('region');

    citySelect.addEventListener('change', function() {
        const selectedCity = this.value;
        const regions = cityRegionMap[selectedCity] || [];
        
        regionSelect.innerHTML = '<option value="">-- City Overview --</option>';
        
        if (regions.length > 0) {
            regionSelect.disabled = false;
            regions.forEach(r => {
                const option = document.createElement('option');
                option.value = r;
                option.textContent = r;
                regionSelect.appendChild(option);
            });
            if (regions.length === 1) {
                regionSelect.value = regions[0];
            }
        } else {
            regionSelect.disabled = true;
        }
    });

    document.getElementById('analyticsForm').addEventListener('submit', async (e) => {
        e.preventDefault();
        const resDiv = document.getElementById('results');
        const cityVal = citySelect.value;
        
        if (!cityVal) {
            alert("Please select a city first!");
            return;
        }

        resDiv.innerHTML = `
            <div class="text-center">
                <div class="loading-spinner"></div>
                <div class="text-info mt-3">📊 Loading analytics...</div>
            </div>
        `;
        
        try {
            const res = await fetch('/api/analytics', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({
                    city: cityVal,
                    region: regionSelect.value
                })
            });
            
            const data = await res.json();
            
            setTimeout(() => {
                if (data.error) {
                    resDiv.innerHTML = `<div class="alert alert-danger">${data.error}</div>`;
                    return;
                }
                
                if (data.type === 'city') {
                    let html = `
                        <div class="card p-4 results-container">
                            <div class="overview-header">
                                <span class="overview-icon">📍</span>
                                <h3>${data.city} Overview</h3>
                            </div>
                            <hr>
                            
                            <div class="stat-card">
                                <h5>💰 Average Price</h5>
                                <h2 class="price-color">${data.avg_price}</h2>
                                <small>Median: ${data.median_price}</small><br>
                                <small>Range: ${data.price_range}</small>
                            </div>
                            
                            <div class="stat-card">
                                <h5>📏 Average Area</h5>
                                <h2 class="area-color">${data.avg_area}</h2>
                            </div>
                            
                            <div class="stat-card">
                                <h5>🏠 Most Common Type</h5>
                                <h2 class="type-color">${data.top_property_type}</h2>
                                <small>Total properties: ${data.total_properties}</small>
                            </div>
                            
                            <h5 class="mt-4">Property Distribution:</h5>
                            <ul>`;
                    
                    for (let [type, count] of Object.entries(data.property_distribution)) {
                        html += `<li>${type}: ${count} properties</li>`;
                    }
                    
                    html += `</ul>`;
                    
                    if (data.hotspots.length > 0) {
                        html += `<h5 class="mt-3 hotspot">🔥 Hotspots (Above City Avg):</h5><ul>`;
                        data.hotspots.forEach(h => html += `<li>${h}</li>`);
                        html += `</ul>`;
                    }
                    
                    if (data.coldspots.length > 0) {
                        html += `<h5 class="mt-3 coldspot">❄️ Coldspots (Below City Avg):</h5><ul>`;
                        data.coldspots.forEach(c => html += `<li>${c}</li>`);
                        html += `</ul>`;
                    }
                    
                    html += `</div>`;
                    resDiv.innerHTML = html;
                    
                } else {
                    let statusColor = data.status === 'Hotspot' ? 'hotspot' : (data.status === 'Coldspot' ? 'coldspot' : '');
                    let statusIcon = data.status === 'Hotspot' ? '🔥' : (data.status === 'Coldspot' ? '❄️' : '⚖️');

                    let html = `
                        <div class="card p-4 results-container">
                            <div class="overview-header">
                                <span class="overview-icon">${statusIcon}</span>
                                <div>
                                    <h3>${data.region}, ${data.city}</h3>
                                    <h4 class="${statusColor}">${data.status} Area</h4>
                                </div>
                            </div>
                            <small style="color: #a09080; font-weight: 500;">Compared to ${data.city} average</small>
                            <hr>
                            
                            <div class="stat-card">
                                <h5>💰 Average Price</h5>
                                <h2 class="price-color">${data.avg_price}</h2>
                                <small>Median: ${data.median_price}</small>
                            </div>
                            
                            <div class="stat-card">
                                <h5>📏 Average Area</h5>
                                <h2 class="area-color">${data.avg_area}</h2>
                            </div>
                            
                            <div class="stat-card">
                                <h5>🏠 Most Common Type</h5>
                                <h2 class="type-color">${data.top_property_type}</h2>
                                <small>Total: ${data.total_properties} properties</small>
                            </div>
                            
                            <h5 class="mt-4">Property Distribution:</h5>
                            <ul>`;
                    
                    for (let [type, count] of Object.entries(data.property_distribution)) {
                        html += `<li>${type}: ${count} properties</li>`;
                    }
                    
                    html += `</ul></div>`;
                    resDiv.innerHTML = html;
                }
            }, 300);
            
        } catch (err) {
            resDiv.innerHTML = `<div class="alert alert-danger">Error: ${err.message}</div>`;
        }
    });
</script>
</body>
</html>

Overwriting templates/analytics.html


In [27]:
!python app.py

Both models loaded successfully!
Unified App running on http://0.0.0.0:5000
Models: Recommender + Analytics
 * Serving Flask app 'app'
 * Debug mode: on
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.22.0.8:5000
Press CTRL+C to quit
 * Restarting with stat
Both models loaded successfully!
Unified App running on http://0.0.0.0:5000
Models: Recommender + Analytics
 * Debugger is active!
 * Debugger PIN: 121-802-227
172.22.0.1 - - [25/Nov/2025 22:56:59] "GET / HTTP/1.1" 200 -
172.22.0.1 - - [25/Nov/2025 22:57:02] "GET /analytics HTTP/1.1" 200 -
172.22.0.1 - - [25/Nov/2025 22:57:10] "POST /api/analytics HTTP/1.1" 200 -
172.22.0.1 - - [25/Nov/2025 22:59:10] "GET / HTTP/1.1" 200 -
172.22.0.1 - - [25/Nov/2025 23:00:31] "GET /analytics HTTP/1.1" 200 -
172.22.0.1 - - [25/Nov/2025 23:00:35] "POST /api/analytics HTTP/1.1" 200 -
^C
